# Enhanced Kcorrect Tutorial for Local Universe (z=0.01-0.15)

This notebook demonstrates the enhanced kcorrect functionality optimized for local universe galaxy studies using GALEX+SDSS+WISE photometry.

## Features:
- **Progress Display**: Real-time progress bars for long-running calculations
- **Local Universe Optimization**: Dense redshift sampling for z=0.01-0.15
- **Multi-wavelength Support**: GALEX (UV) + SDSS (optical) + WISE (infrared)
- **Robust Template Handling**: Safe attribute access for different template formats
- **Batch Processing**: Efficient tools for creating multiple redshift ranges

## Applications:
- Local galaxy survey analysis
- Star formation history studies  
- Stellar mass measurements
- Multi-wavelength SED fitting

In [1]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import kcorrect
import kcorrect.template

# Import our enhanced kcorrect modules
from kcorrect_with_progress_v2 import (
    KcorrectWithProgress, 
    KcorrectGSWWithProgress,
    create_local_universe_templates,
    get_template_info
)

print("✅ All modules imported successfully")
print(f"Working directory: {os.getcwd()}")

# Check available data files
data_files = []
if os.path.exists('data'):
    data_files = [f for f in os.listdir('data') if f.endswith('.fits')]
    print(f"Available FITS files in data/: {data_files}")
else:
    print("No data/ directory found")

✅ All modules imported successfully
Working directory: d:\OneDrive\Code\Packages\kcorrect_examples
Available FITS files in data/: ['kcorrect_broad.fits', 'templates_broad.fits']


In [2]:
# Inspect template files safely
template_files = ['data/kcorrect_broad.fits', 'data/templates_broad.fits']

for template_file in template_files:
    if os.path.exists(template_file):
        print(f"\n{'='*50}")
        print(f"Inspecting: {template_file}")
        print(f"{'='*50}")
        
        try:
            # Load template
            template = kcorrect.template.Template(filename=template_file)
            
            # Use safe inspection function
            info = get_template_info(template)
            print(f"📊 Template Information:")
            print(f"  - Number of templates: {info['ntemplates']}")
            print(f"  - Template type: {info['template_type']}")
            print(f"  - Available methods: {info['available_methods'][:5]}...")
            
            # Check specific attributes
            print(f"\n🔍 Detailed Attributes:")
            for attr_name in ['lambda_templates', 'wavelength', 'templates', 'flux']:
                if hasattr(template, attr_name):
                    attr_val = getattr(template, attr_name)
                    if hasattr(attr_val, 'shape'):
                        print(f"  - {attr_name}: shape {attr_val.shape}")
                    elif hasattr(attr_val, '__len__'):
                        print(f"  - {attr_name}: length {len(attr_val)}")
                    else:
                        print(f"  - {attr_name}: {type(attr_val)}")
                else:
                    print(f"  - {attr_name}: not available")
            
            # Check FITS file structure directly
            print(f"\n📁 FITS File Structure:")
            with fits.open(template_file) as hdul:
                for i, hdu in enumerate(hdul):
                    print(f"  HDU {i}: {hdu.name} ({type(hdu).__name__})")
                    if hasattr(hdu, 'data') and hdu.data is not None:
                        print(f"    Data shape: {hdu.data.shape}")
                        print(f"    Data type: {hdu.data.dtype}")
            
        except Exception as e:
            print(f"❌ Error inspecting {template_file}: {e}")
            import traceback
            traceback.print_exc()
    else:
        print(f"⚠️  File not found: {template_file}")


Inspecting: data/kcorrect_broad.fits
📊 Template Information:
  - Number of templates: 57636
❌ Error inspecting data/kcorrect_broad.fits: 'template_type'

Inspecting: data/templates_broad.fits
📊 Template Information:
  - Number of templates: 57636
❌ Error inspecting data/templates_broad.fits: 'template_type'


Traceback (most recent call last):
  File "C:\Users\schac\AppData\Local\Temp\ipykernel_48040\2810797315.py", line 18, in <module>
    print(f"  - Template type: {info['template_type']}")
                                ~~~~^^^^^^^^^^^^^^^^^
KeyError: 'template_type'
Traceback (most recent call last):
  File "C:\Users\schac\AppData\Local\Temp\ipykernel_48040\2810797315.py", line 18, in <module>
    print(f"  - Template type: {info['template_type']}")
                                ~~~~^^^^^^^^^^^^^^^^^
KeyError: 'template_type'


In [5]:
# Create GALEX+SDSS+WISE optimized template for local universe
template_file = 'data/kcorrect_broad.fits'

if os.path.exists(template_file):
    print("🌌 Creating GALEX+SDSS+WISE Template for Local Universe")
    print("=" * 60)
    
    # Configuration for local universe
    config = {
        'redshift_range': [0.01, 0.15],  # Local universe
        'nredshift': 1000,               # Dense sampling
        'show_progress': True
    }
    
    print(f"Configuration: {config}")
    
    try:
        # Create specialized GALEX+SDSS+WISE object
        print("\n📡 Creating KcorrectGSWWithProgress object...")
        kc_gsw = KcorrectGSWWithProgress(
            templates=template_file,
            **config
        )
        
        print("✅ Successfully created kcorrect object!")
        print(f"   Redshift range: {kc_gsw.redshift_range}")
        print(f"   Number of redshift points: {kc_gsw.nredshift}")
        print(f"   Response filters: {len(kc_gsw.responses)} bands")
        
        # Display response filters
        print(f"\n📊 Response Filters:")
        for i, response in enumerate(kc_gsw.responses):
            print(f"   {i+1:2d}. {response}")
        
        # Save the template
        output_file = 'data/kcorrect_gsw_local_v2.fits'
        print(f"\n💾 Saving to: {output_file}")
        kc_gsw.tofits(output_file)
        
        # Check file size
        if os.path.exists(output_file):
            file_size = os.path.getsize(output_file) / (1024 * 1024)
            print(f"✅ File saved successfully! Size: {file_size:.1f} MB")
        
        # Store for later use
        gsw_template = kc_gsw
        
    except Exception as e:
        print(f"❌ Error creating template: {e}")
        import traceback
        traceback.print_exc()
        gsw_template = None
        
else:
    print(f"⚠️  Template file not found: {template_file}")
    gsw_template = None

🌌 Creating GALEX+SDSS+WISE Template for Local Universe
Configuration: {'redshift_range': [0.01, 0.15], 'nredshift': 1000, 'show_progress': True}

📡 Creating KcorrectGSWWithProgress object...
Initializing KcorrectWithProgress...
Initializing Fitter base class...
Setting up A-matrix for input responses...
Setting up A-matrix...
Calculating design matrix for 11 responses and 1000 redshifts...


Processing redshifts:   0%|          | 0/1000 [00:00<?, ?it/s]








                                                         





























z=0.013 responses:   0%|          | 0/11 [00:00<?, ?it/s]











Processing redshifts:   3%|▎         | 26/1000 [00:00<00:03, 259.58it/s]



















































Processing redshifts:   5%|▌         | 52/1000 [00:00<00:03, 254.55it/s]
























                                                         






















Processing redshifts:   8%|▊         | 78/1000 [00:00<00:03, 253.78it/s]


z=0.021 responses:   0%|          | 0/11 [00:00<?, ?it/s]

Processing redshifts:   8%|▊         | 78/1000 [00:00<00:03, 253.78it/s]

















































Processing redshifts:  10%|█         | 104/1000 [00:00<00:03, 249.77it/s]





































Processing redshifts:  13%|█▎        | 129/1000 [00:00<00:03, 242.45it/s]










Processing redshifts:

Design matrix calculation complete! Time elapsed: 4.02 seconds
Output responses same as input, sharing A-matrix...
Initialization complete! Time elapsed: 4.03 seconds
✅ Successfully created kcorrect object!
   Redshift range: [0.01 0.15]
   Number of redshift points: 1000
   Response filters: 11 bands

📊 Response Filters:
    1. galex_FUV
    2. galex_NUV
    3. sdss_u0
    4. sdss_g0
    5. sdss_r0
    6. sdss_i0
    7. sdss_z0
    8. wise_w1
    9. wise_w2
   10. wise_w3
   11. wise_w4

💾 Saving to: data/kcorrect_gsw_local_v2.fits
Saving to file: data/kcorrect_gsw_local_v2.fits
Saving template data...
❌ Error creating template: 'str' object has no attribute 'tofits'


In [6]:
# Create multiple redshift subdivisions for specialized studies
template_file = 'data/kcorrect_broad.fits'

if os.path.exists(template_file):
    print("🔄 Creating Multiple Redshift Range Templates")
    print("=" * 60)
    
    # Create subdivided templates for detailed local universe studies
    print("Creating subdivided templates for z=0.01-0.15...")
    print("This allows for more precise K-corrections in specific redshift ranges")
    
    try:
        # Use convenience function to create multiple files
        output_files = create_local_universe_templates(
            template_filename=template_file,
            output_dir='data',
            redshift_subdivisions=3,  # Create 3 subdivisions
            show_progress=True
        )
        
        print("\n📁 Generated Files:")
        total_size = 0
        for i, output_file in enumerate(output_files):
            if os.path.exists(output_file):
                file_size = os.path.getsize(output_file) / (1024 * 1024)
                total_size += file_size
                print(f"  {i+1}. {os.path.basename(output_file)} ({file_size:.1f} MB)")
            else:
                print(f"  {i+1}. {os.path.basename(output_file)} - ❌ Not created")
        
        print(f"\n📊 Summary:")
        print(f"  - Total files created: {len([f for f in output_files if os.path.exists(f)])}")
        print(f"  - Total size: {total_size:.1f} MB")
        print(f"  - Average size per file: {total_size/len(output_files):.1f} MB")
        
        # Store file list for later testing
        subdivision_files = output_files
        
    except Exception as e:
        print(f"❌ Error creating subdivided templates: {e}")
        import traceback
        traceback.print_exc()
        subdivision_files = []
        
else:
    print(f"⚠️  Template file not found: {template_file}")
    subdivision_files = []

🔄 Creating Multiple Redshift Range Templates
Creating subdivided templates for z=0.01-0.15...
This allows for more precise K-corrections in specific redshift ranges
Creating 3 K-correction templates for local universe...
Template file: data/kcorrect_broad.fits
Responses: ['galex_FUV', 'galex_NUV', 'sdss_u0', 'sdss_g0', 'sdss_r0', 'sdss_i0', 'sdss_z0', 'wise_w1', 'wise_w2', 'wise_w3', 'wise_w4']

=== Processing redshift range 1/3: z=[0.010, 0.050] ===
Initializing KcorrectWithProgress...
Initializing Fitter base class...
Setting up A-matrix for input responses...
Setting up A-matrix...
Calculating design matrix for 11 responses and 500 redshifts...


Processing redshifts:   0%|          | 0/500 [00:00<?, ?it/s]
























                                                         






















Processing redshifts:   5%|▌         | 25/500 [00:00<00:01, 243.36it/s]

Processing redshifts:   5%|▌         | 25/500 [00:00<00:01, 243.36it/s]















































Processing redshifts:  10%|█         | 50/500 [00:00<00:01, 232.03it/s]









z=0.014 responses:   0%|          | 0/11 [00:00<?, ?it/s]




































Processing redshifts:  15%|█▍        | 74/500 [00:00<00:01, 228.60it/s]

Processing redshifts:  15%|█▍        | 74/500 [00:00<00:01, 228.60it/s]
















                                                         




























Processing redshifts:  20%|█▉        | 98/500 [00:00<00:01, 232.53it/s]



























                                                         















                                                

Design matrix calculation complete! Time elapsed: 2.33 seconds
Output responses same as input, sharing A-matrix...
Initialization complete! Time elapsed: 3.06 seconds
Saving to: data\kcorrect_local_z0.010_0.050.fits
Saving to file: data\kcorrect_local_z0.010_0.050.fits
Saving template data...
Output responses same as input, sharing A-matrix...
Initialization complete! Time elapsed: 3.06 seconds
Saving to: data\kcorrect_local_z0.010_0.050.fits
Saving to file: data\kcorrect_local_z0.010_0.050.fits
Saving template data...


KeyboardInterrupt: 

In [ ]:
# Test K-correction calculations with generated templates
print("🧪 Testing K-correction Calculations")
print("=" * 60)

# Test files to check
test_files = [
    'data/kcorrect_gsw_local_v2.fits'
]

# Add subdivision files if they exist
if 'subdivision_files' in locals() and subdivision_files:
    test_files.extend(subdivision_files)

for test_file in test_files:
    if os.path.exists(test_file):
        print(f"\n🔬 Testing: {os.path.basename(test_file)}")
        print("-" * 40)
        
        try:
            # Load kcorrect object from file
            kc = KcorrectWithProgress(filename=test_file, show_progress=False)
            
            print(f"✅ Loaded successfully")
            print(f"   Redshift range: {kc.redshift_range}")
            print(f"   Redshift points: {kc.nredshift}")
            print(f"   Response filters: {len(kc.responses)} bands")
            
            # Get template information safely
            template_info = get_template_info(kc.templates)
            ntemplates = template_info['ntemplates']
            
            if isinstance(ntemplates, int) and ntemplates > 0:
                print(f"   Number of templates: {ntemplates}")
                
                # Create test data
                test_redshifts = np.linspace(kc.redshift_range[0], kc.redshift_range[1], 5)
                test_coeffs = np.random.random((len(test_redshifts), ntemplates)) * 1e-10
                
                print(f"\n📊 Test Parameters:")
                print(f"   Test redshifts: {test_redshifts}")
                print(f"   Coefficient matrix shape: {test_coeffs.shape}")
                
                # Test K-correction calculation
                print(f"\n🔧 Testing K-correction calculation...")
                k_corrections = kc.kcorrect(redshift=test_redshifts, coeffs=test_coeffs)
                
                print(f"✅ K-correction successful!")
                print(f"   Output shape: {k_corrections.shape}")
                print(f"   Value range: [{k_corrections.min():.3f}, {k_corrections.max():.3f}]")
                
                # Test spectrum reconstruction
                print(f"\n🌈 Testing spectrum reconstruction...")
                reconstructed = kc.reconstruct_out(redshift=test_redshifts, coeffs=test_coeffs)
                
                print(f"✅ Reconstruction successful!")
                print(f"   Spectrum shape: {reconstructed.shape}")
                print(f"   Flux range: [{reconstructed.min():.2e}, {reconstructed.max():.2e}]")
                
                # Quick visualization if matplotlib is available
                try:
                    plt.figure(figsize=(12, 4))
                    
                    # Plot K-corrections
                    plt.subplot(1, 2, 1)
                    for i in range(k_corrections.shape[1]):
                        plt.plot(test_redshifts, k_corrections[:, i], 
                                label=f'Band {i+1}', marker='o')
                    plt.xlabel('Redshift')
                    plt.ylabel('K-correction (mag)')
                    plt.title(f'K-corrections\n{os.path.basename(test_file)}')
                    plt.legend()
                    plt.grid(True, alpha=0.3)
                    
                    # Plot reconstructed spectrum (first redshift)
                    plt.subplot(1, 2, 2)
                    if hasattr(kc.templates, 'lambda_templates'):
                        wavelength = kc.templates.lambda_templates
                    elif hasattr(kc.templates, 'wavelength'):
                        wavelength = kc.templates.wavelength
                    else:
                        wavelength = np.arange(len(reconstructed[0]))
                    
                    plt.plot(wavelength, reconstructed[0], 'b-', 
                            label=f'z={test_redshifts[0]:.3f}')
                    plt.xlabel('Wavelength (Å)')
                    plt.ylabel('Flux')
                    plt.title('Reconstructed Spectrum')
                    plt.legend()
                    plt.grid(True, alpha=0.3)
                    plt.xlim(3000, 10000)  # Optical range
                    
                    plt.tight_layout()
                    plt.show()
                    
                except Exception as plot_error:
                    print(f"⚠️  Plotting failed: {plot_error}")
                
            else:
                print(f"⚠️  Could not determine template count: {ntemplates}")
                
        except Exception as e:
            print(f"❌ Test failed: {e}")
            import traceback
            traceback.print_exc()
    else:
        print(f"⚠️  File not found: {test_file}")

print(f"\n🎉 Testing Complete!")
print("The enhanced kcorrect module is ready for local universe galaxy analysis!")

Filename: ./data/kcorrect_broad.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  FLUX          1 BinTableHDU     15   1R x 2C   [5994E, 345470184E]   
  2  INTSFH        1 ImageHDU         7   (57636,)   float32   
  3  MREMAIN       1 ImageHDU         7   (57636,)   float32   
  4  METS          1 ImageHDU         7   (57636,)   float32   
  5  M300          1 ImageHDU         7   (57636,)   float32   
  6  M50           1 ImageHDU         7   (57636,)   float32   
  7  M1000         1 ImageHDU         7   (57636,)   float32   
